In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import numpy as np 
import pandas as pd
from keras import optimizers 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
#from kerastuner.tuners import RandomSearch
#from kerastuner.engine.hyperparameters import HyperParameters
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
from keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import cv2

Using TensorFlow backend.


In [3]:
print(os.listdir("../input"))

['dogs-vs-cats']


data pre-processing

In [4]:
filenames = os.listdir("../input/dogs-vs-cats/train/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})


In [6]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 

In [7]:

df_train, df_valid = train_test_split(df, test_size=0.20, random_state=42)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [8]:
df_train.shape[0]

20000

setting the batch size to 100

In [9]:
train_size = df_train.shape[0]
valid_size = df_valid.shape[0]
batch_size = 100

generate image input available for model 

In [10]:
# produce image with 15 different angles to have better training materials
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    df_train, 
    "../input/dogs-vs-cats/train/train/", 
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode="categorical",
    batch_size=batch_size
)

Found 20000 validated image filenames belonging to 2 classes.


In [11]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    df_valid, 
    "../input/dogs-vs-cats/train/train/", 
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode="categorical",
    batch_size=batch_size
)

Found 5000 validated image filenames belonging to 2 classes.


In [12]:
example_df = df_train.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    "../input/dogs-vs-cats/train/train/", 
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode='categorical'
)

Found 1 validated image filenames belonging to 1 classes.


building a model, which I selected the constructed model VGG-16

In [14]:
model = VGG16(include_top=True, weights='imagenet')
#input_shape=(64, 64, 3),
#model.layers.pop()

model.summary()




553467904/553467096 [==============================] - 20s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

In [15]:
model.inputs

[<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>]

In [16]:
model.outputs

[<tf.Tensor 'predictions/Softmax:0' shape=(None, 1000) dtype=float32>]

change the output layer and add two more Dense layers to increase the accuracy of the models.(from try and errors :)) 

In [17]:
x = model.layers[-2].output

#model.layers[-1].outbound_nodes = []

x1 = Dense(10, activation = 'relu')(x)

predictions=Dense(2, activation='softmax')(x1)

for layer in model.layers[:13]:

       layer.trainable = False

model=Model(model.input,predictions)

In [18]:
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

#rmspop = RMSprop(learning_rate=0.0001, rho=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [19]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [20]:
history = model.fit_generator(
    train_generator, 
    epochs=3,
    validation_data=validation_generator,
    validation_steps=valid_size//batch_size,
    steps_per_epoch=train_size//batch_size
)

Epoch 1/3
200/200 [==============================] - 291s 1s/step - loss: 0.3177 - accuracy: 0.8616 - val_loss: 0.0535 - val_accuracy: 0.9576
Epoch 2/3
200/200 [==============================] - 269s 1s/step - loss: 0.1083 - accuracy: 0.9567 - val_loss: 0.0976 - val_accuracy: 0.9690
Epoch 3/3
200/200 [==============================] - 271s 1s/step - loss: 0.0872 - accuracy: 0.9656 - val_loss: 0.1333 - val_accuracy: 0.9698


In [21]:
test_filenames = os.listdir("../input/dogs-vs-cats/test1/test1")
df_test = pd.DataFrame({
    'filename': test_filenames
})
test_size = df_test.shape[0]

In [22]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    df_test, 
    "../input/dogs-vs-cats/test1/test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(224,224),
    batch_size=batch_size,
    shuffle=False
)

Found 12500 validated image filenames.


In [23]:
predict = model.predict_generator(test_generator, steps=np.ceil(test_size/batch_size))

In [26]:
df_test['category'] = predict[:,1]

In [27]:
df_test

,filename,category
0,1258.jpg,0.000800
1,12337.jpg,0.980510
2,9506.jpg,0.000007
3,9773.jpg,0.003092
4,3522.jpg,0.997084
...,...,...
12495,12450.jpg,0.000200
12496,9119.jpg,0.999494
12497,1970.jpg,0.542462
12498,6184.jpg,0.004791


In [28]:
submission_df = df_test.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)